<a href="https://colab.research.google.com/github/Ajay1082158t/FMML_labs/blob/main/Copy_of_Module_01_Lab_02_MLPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning terms and metrics

FMML Module 1, Lab 2<br>


 In this lab, we will show a part of the ML pipeline by extracting features, training and testing

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=42)

In this lab, we will use the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district.

Let us download and examine the dataset.

In [ ]:
 dataset =  datasets.fetch_california_housing()
 # print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
 # print(dataset.keys())  # if you want to know what else is there in this dataset
 dataset.target = dataset.target.astype(np.int) # so that we can classify
 print(dataset.data.shape)
 print(dataset.target.shape)

(20640, 8)
(20640,)


<ipython-input-2-60ae2e9a125e>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataset.target = dataset.target.astype(np.int) # so that we can classify


Here is a function for calculating the 1-nearest neighbours

In [ ]:
def NN1(traindata, trainlabel, query):
  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here
  sq = diff*diff # square the differences
  dist = sq.sum(1) # add up the squares
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [ ]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm.

In [ ]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

Let us make a function to split the dataset with the desired probability.

In [ ]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [ ]:
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4144
Number of other samples =  16496
Percent of test data =  20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [ ]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)

What is the accuracy of our classifiers on the train dataset?

In [ ]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

Train accuracy using nearest neighbour is  1.0
Train accuracy using random classifier is  0.164375808538163


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case.

Let us predict the labels for our validation set and get the accuracy

In [ ]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy using nearest neighbour is  0.34108527131782945
Validation accuracy using random classifier is  0.1688468992248062


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier.

Now let us try another random split and check the validation accuracy

In [ ]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.34048257372654156


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [ ]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

Test accuracy is  0.34917953667953666


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute.

In [ ]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [ ]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.33584635395170215
test accuracy is  0.34917953667953666


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


ANSWER FOR QUESTION NO 1:-
Reduced Variance: When you use a single random split of your data into training and validation sets, the performance of your model can be heavily influenced by the specific data points in that split. If you were unlucky and ended up with an atypical validation set, your accuracy score could be misleadingly high or low. By performing multiple splits and averaging the results, you reduce the impact of such randomness and get a more stable estimate of your model's performance.

Better Representation: Each fold in k-fold cross-validation serves as both a training and validation set. This means that your model is trained and evaluated on different subsets of your data, providing a more representative assessment of how well your model generalizes to different parts of the dataset.

Robustness to Outliers or Anomalies: In a single split, if your data contains outliers or anomalies that happen to be in the validation set, it can significantly affect the validation accuracy. By averaging over multiple splits, the influence of such outliers is diminished.

In summary, averaging validation accuracy across multiple splits enhances the consistency and reliability of your model evaluation, making it a standard practice in machine learning to obtain a more accurate assessment of your model's performance.







ANSWER FOR QUESTION NO 2:-
Validation Accuracy: This is the accuracy metric you calculate during the training process to assess how well your model is performing on a dataset that it hasn't seen during training. It helps you make decisions about model hyperparameters, architecture, or training strategies.

Test Accuracy: This is the metric you calculate on a completely separate dataset that your model has never encountered during training or validation. Test accuracy provides an estimate of how well your model generalizes to real-world, unseen data.

Using k-fold cross-validation or other validation techniques to estimate model performance helps you obtain a more accurate estimate of how well your model might perform on the test data. It provides a robust and unbiased assessment of your model's generalization ability because it evaluates the model on multiple different subsets of the data.

While it doesn't directly give you the test accuracy (which is only determined when you evaluate the model on the actual test dataset), it does provide a more reliable indicator of how your model will likely perform on unseen data compared to a single validation split. This estimate is crucial for model selection, hyperparameter tuning, and understanding your model's expected performance in practice.

In practice, you would still need to evaluate your final model on the separate test dataset to obtain the actual test accuracy, but the estimate obtained through cross-validation is a valuable step in assessing your model's readiness for deployment.


ANSWER FOR QUESTION NO 3:-
Higher Iterations (Higher "k"): When you increase the number of iterations in k-fold cross-validation, such as using a larger value of "k," you are essentially dividing your dataset into more subsets (folds) and performing more iterations of training and validation. This can have several effects:

Better Estimate: With higher iterations, you are using more diverse subsets of your data for validation, which tends to provide a more accurate estimate of your model's performance. It reduces the impact of randomness in the selection of validation sets and can yield a more stable and reliable estimate.

Longer Computation Time: On the downside, increasing the number of iterations also increases the computational cost because you are training and evaluating your model multiple times. This might be a concern, especially with large datasets or complex models.

Lower Iterations (Lower "k"): Conversely, if you decrease the number of iterations (using a smaller "k"), you will have fewer validation sets, which can lead to the following effects:

Higher Variance in Estimate: With fewer iterations, your estimate of model performance may have higher variance. The validation results can be more sensitive to the specific random splits of the data, which may lead to less stable estimates.

Faster Computation: On the positive side, fewer iterations reduce computational time since you are training and validating your model less frequently.

In summary, the number of iterations in k-fold cross-validation is a trade-off between the accuracy and stability of your performance estimate and the computational cost. A higher number of iterations generally provides a better estimate of your model's performance but comes with increased computation time. However, there is typically a point of diminishing returns where further increasing "k" doesn't yield significant improvements in the estimate. Common values for "k" in practice are 5 or 10, as they strike a balance between accuracy and computational efficiency.

Ultimately, the choice of the number of iterations should be based on your specific problem, available computational resources, and the need for a reliable estimate of your model's performance.

ANSWER FOR QUESTION NO 4:-
Small Training Dataset: If your training dataset is very small, increasing the number of iterations in cross-validation won't magically create more data. While it can provide a more stable estimate of how well your model performs on the available data, it cannot overcome the limitations imposed by a lack of sufficient training data. In such cases, the model may struggle to learn complex patterns, leading to poor generalization performance.

Small Validation Dataset: Similarly, if your validation dataset is very small, increasing the number of iterations won't make the validation dataset larger. A small validation dataset can still lead to high variability in your cross-validation results, as the performance estimates will be influenced by the specific data points in the validation split. This can make it challenging to draw reliable conclusions about your model's performance.

In scenarios where you have very limited training data or a very small validation dataset, you should consider alternative strategies:

Data Augmentation: If possible, use data augmentation techniques to artificially increase the effective size of your training dataset. Data augmentation involves creating new training examples by applying random transformations to your existing data.

Transfer Learning: Utilize pre-trained models on larger, related datasets as a starting point. Fine-tune these models on your small dataset to leverage the knowledge learned from the larger dataset.

Simpler Models: Consider using simpler, more regularized models with fewer parameters that are less prone to overfitting when training data is limited.

Collect More Data: If feasible, collect more data to increase the size of your training dataset. This is often the most effective way to improve model performance.

While increasing the number of iterations in cross-validation is valuable for assessing and optimizing model performance, it cannot compensate for a severe lack of data. It's important to address the data quantity issue directly through techniques like data augmentation, transfer learning, or data collection when dealing with very small datasets.